In [95]:
import tweepy
import json
import re

class MyStreamListener(tweepy.StreamListener):
    
    def __init__(self, api=None):
        super(MyStreamListener, self).__init__()
        self.num_tweets = 2018
        self.count = 1
        self.all_tweets = ""
        self.all_tweets_list = []
    
    def on_status(self, status):
        print(status.text)
        
    def on_data(self, data):
        tweet = json.loads(data)
        if not tweet['retweeted'] and 'RT @' not in tweet['text']:
            if self.count == (self.num_tweets+1):
                #print(self.all_tweets)
                #print(self.all_tweets_list)
                with open('data.json', 'w') as outfile:
                    json.dump(self.all_tweets_list, outfile)
                return False
            t = dict()
            t['id'] = self.count
            emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
            tweet['text'] = emoji_pattern.sub(r'', tweet['text'])
            t['text'] = tweet['text']
            self.count += 1
            self.all_tweets += (json.dumps(t)+"\n")
            self.all_tweets_list.append(t)
            #print(t)
        return True
    
    def on_error(self, status_code):
        if status_code == 420:
            return False
        
    def get_all_tweets(self):
        return self.all_tweets
        
ACCESS_TOKEN = '1040311982015037440-1TSqPfFZmXn1fUS1qplchjVbWBMANn'
ACCESS_SECRET = 'K0efO3na1D9Mw1Du7rkROzgOW8zDTLTG6AcVTuLqMzjQq'
CONSUMER_KEY = 'j4kLoamWfoPbw72VLoQwjmzeF'
CONSUMER_SECRET = 'UwEGlKxK2xX4I59FXhwUmc1A4wPoW2ZceTY1ZI2iTdEGDOaOjY'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.secure = True
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tweepy.API(auth)

myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=MyStreamListener())
myStream.filter(track=['lol'])


In [96]:
import pyspark 
from pyspark.sql import SQLContext

spark = SparkSession.builder.config(conf=SparkConf()).getOrCreate()
sc = SparkContext.getOrCreate()
sqlc = SQLContext(sc)
df = sqlc.read.json("data.json",multiLine=True)
df.show()

+---+--------------------+
| id|                text|
+---+--------------------+
|  1|@cenkuygur @polit...|
|  2|@SplashIQ A: i do...|
|  3|     I’m excited lol|
|  4|@CurlsAndSports S...|
|  5|//Sorry but this ...|
|  6|       Lol irritated|
|  7|@Sabree____ I don...|
|  8|Lol nigga dropped...|
|  9|@ConnectionEnded ...|
| 10|            Stop lol|
| 11|This will have me...|
| 12|GOD DAMMIT! It to...|
| 13|@MandJTV_Michael ...|
| 14|AlL oF tHe oLd Ra...|
| 15|Me and the Dean o...|
| 16|Lol there was a p...|
| 17|@steph62490 eh ou...|
| 18|Click ..click... ...|
| 19|@Soddem_666 Looks...|
| 20|I worked out too ...|
+---+--------------------+



In [ ]:
# The code was removed by Watson Studio for sharing.

In [75]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator
from pyspark.sql.functions import lit
from pyspark.sql import types

df_data_2 = df_data_2.withColumn("label", df_data_2.label.cast("Int"))
#train = df_data_2
(train, test) = df_data_2.randomSplit([0.9, 0.1], seed = 1000)

tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=20,regParam=0.001)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

# Fit the pipeline to training documents.
model= pipeline.fit(train)

# Make predictions on test documents and print columns of interest.
predictions = model.transform(test)
#predictions.show()

evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
print("F1: %g" % (evaluator.evaluate(predictions)))

model.write().overwrite().save("logreg.model")


F1: 0.674556


In [97]:
from pyspark.ml import PipelineModel
pipelineModel = PipelineModel.load("logreg.model")
prediction = pipelineModel.transform(df)
prediction.show()

+---+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
| id|                text|               words|            features|       rawPrediction|         probability|prediction|
+---+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  1|@cenkuygur @polit...|[@cenkuygur, @pol...|(262144,[15889,27...|[2.13603758058881...|[0.21718246030064...|       4.0|
|  2|@SplashIQ A: i do...|[@splashiq, a:, i...|(262144,[9639,132...|[4.21818986059459...|[0.74961251262493...|       0.0|
|  3|     I’m excited lol| [i’m, excited, lol]|(262144,[31950,61...|[1.57457674815598...|[0.08050379155457...|       4.0|
|  4|@CurlsAndSports S...|[@curlsandsports,...|(262144,[8727,201...|[1.87547934277638...|[0.17945818873223...|       4.0|
|  5|//Sorry but this ...|[//sorry, but, th...|(262144,[15889,19...|[3.13322736106635...|[0.55614350596356...|       0.0|
|  6|       Lol irritate

In [98]:
prediction.filter(prediction['prediction'] == 4) \
    .select("id","text","prediction") \
    .orderBy("id", ascending=False) \
    .show(n = 10, truncate = 30)

prediction.filter(prediction['prediction'] == 2) \
    .select("id","text","prediction") \
    .orderBy("id", ascending=False) \
    .show(n = 10, truncate = 30)

prediction.filter(prediction['prediction'] == 0) \
    .select("id","text","prediction",) \
    .orderBy("id", ascending=False) \
    .show(n = 10, truncate = 30)

+---+------------------------------+----------+
| id|                          text|prediction|
+---+------------------------------+----------+
| 19|@Soddem_666 Looks like fara...|       4.0|
| 16|Lol there was a point when ...|       4.0|
| 13|@MandJTV_Michael Only do it...|       4.0|
| 11|This will have me super ner...|       4.0|
| 10|                      Stop lol|       4.0|
|  9|@ConnectionEnded @IFleshMer...|       4.0|
|  8|Lol nigga dropped the mic s...|       4.0|
|  7|@Sabree____ I don’t be seei...|       4.0|
|  6|                 Lol irritated|       4.0|
|  4|@CurlsAndSports Saludos des...|       4.0|
+---+------------------------------+----------+
only showing top 10 rows

+---+------------------------------+----------+
| id|                          text|prediction|
+---+------------------------------+----------+
| 18|        Click ..click... click|       2.0|
| 17|@steph62490 eh oui pour ca ...|       2.0|
+---+------------------------------+----------+

+---+--------